In [1]:
!pip install --upgrade pip
!pip install xgboost


  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\Hp\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import os

# Load the dataset
df = pd.read_csv("combined_pose_data.csv")

# Prepare features and labels
X = df.drop(columns=['Label'])
y = df['Label']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# Check class distribution before split
print("Class distribution in full dataset:", pd.Series(y_encoded).value_counts())

# Split the data with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Check class distribution after split
print("Class distribution in training set:", pd.Series(y_train).value_counts())
print("Class distribution in test set:", pd.Series(y_test).value_counts())

# Train XGBoostClassifier with optimized parameters and early stopping
xgb_model = XGBClassifier(
    learning_rate=0.1,
    max_depth=6,
    n_estimators=200,
    random_state=42,
    early_stopping_rounds=10
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)
xgb_pred = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_report = classification_report(y_test, xgb_pred, target_names=le.classes_, zero_division=0, output_dict=False)
print("XGBoost Accuracy:", xgb_accuracy)
print("XGBoost Report:\n", xgb_report)

# Train RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_report = classification_report(y_test, rf_pred, target_names=le.classes_, zero_division=0, output_dict=False)
print("RandomForest Accuracy:", rf_accuracy)
print("RandomForest Report:\n", rf_report)

# Save the better model and label encoder
best_model = xgb_model if xgb_accuracy > rf_accuracy else rf_model
joblib.dump(best_model, "exercise_classifier_best.pkl")
joblib.dump(le, "label_encoder.pkl")
print("Model saved as exercise_classifier_best.pkl and label encoder as label_encoder.pkl")

# Create result folder if it doesn't exist
result_dir = "result"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# Save accuracy scores to a file
with open(os.path.join(result_dir, "accuracy_scores.txt"), "w") as f:
    f.write(f"XGBoost Accuracy: {xgb_accuracy:.4f}\n")
    f.write(f"RandomForest Accuracy: {rf_accuracy:.4f}\n")

# Save classification reports to files
with open(os.path.join(result_dir, "xgboost_report.txt"), "w") as f:
    f.write("XGBoost Classification Report:\n")
    f.write(xgb_report)
with open(os.path.join(result_dir, "randomforest_report.txt"), "w") as f:
    f.write("RandomForest Classification Report:\n")
    f.write(rf_report)

# Plot accuracy metrics
plt.figure(figsize=(8, 6))
models = ['XGBoost', 'RandomForest']
accuracies = [xgb_accuracy, rf_accuracy]
plt.bar(models, accuracies, color=['blue', 'green'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # Set y-axis limit from 0 to 1
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center')
plt.savefig(os.path.join(result_dir, "accuracy_comparison.png"))
plt.close()
print(f"Graph saved as {os.path.join(result_dir, 'accuracy_comparison.png')}")

FileNotFoundError: [Errno 2] No such file or directory: 'combined_pose_data.csv'

In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import os

# Load the dataset
df = pd.read_csv("combined_pose_data.csv")

# Prepare features and labels
X = df.drop(columns=['Label'])
y = df['Label']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# Check class distribution before split
print("Class distribution in full dataset:", pd.Series(y_encoded).value_counts())

# Split the data with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Check class distribution after split
print("Class distribution in training set:", pd.Series(y_train).value_counts())
print("Class distribution in test set:", pd.Series(y_test).value_counts())

# Train XGBoostClassifier with optimized parameters and early stopping
xgb_model = XGBClassifier(
    learning_rate=0.1,
    max_depth=6,
    n_estimators=200,
    random_state=42,
    early_stopping_rounds=10
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)
xgb_pred = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_report = classification_report(y_test, xgb_pred, target_names=le.classes_, zero_division=0, output_dict=False)
print("XGBoost Accuracy:", xgb_accuracy)
print("XGBoost Report:\n", xgb_report)

# Generate and plot confusion matrix for XGBoost
xgb_cm = confusion_matrix(y_test, xgb_pred)
xgb_cm_display = ConfusionMatrixDisplay(confusion_matrix=xgb_cm, display_labels=le.classes_)
plt.figure(figsize=(10, 8))
xgb_cm_display.plot(cmap='Blues')
plt.title('XGBoost Confusion Matrix')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join('result', 'xgboost_confusion_matrix.png'))
plt.close()
print(f"XGBoost confusion matrix saved as {os.path.join('result', 'xgboost_confusion_matrix.png')}")

# Train RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_report = classification_report(y_test, rf_pred, target_names=le.classes_, zero_division=0, output_dict=False)
print("RandomForest Accuracy:", rf_accuracy)
print("RandomForest Report:\n", rf_report)

# Generate and plot confusion matrix for RandomForest
rf_cm = confusion_matrix(y_test, rf_pred)
rf_cm_display = ConfusionMatrixDisplay(confusion_matrix=rf_cm, display_labels=le.classes_)
plt.figure(figsize=(10, 8))
rf_cm_display.plot(cmap='Blues')
plt.title('RandomForest Confusion Matrix')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join('result', 'randomforest_confusion_matrix.png'))
plt.close()
print(f"RandomForest confusion matrix saved as {os.path.join('result', 'randomforest_confusion_matrix.png')}")

# Save the better model and label encoder
best_model = xgb_model if xgb_accuracy > rf_accuracy else rf_model
joblib.dump(best_model, "exercise_classifier_best.pkl")
joblib.dump(le, "label_encoder.pkl")
print("Model saved as exercise_classifier_best.pkl and label encoder as label_encoder.pkl")

# Create result folder if it doesn't exist
result_dir = "result"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# Save accuracy scores to a file
with open(os.path.join(result_dir, "accuracy_scores.txt"), "w") as f:
    f.write(f"XGBoost Accuracy: {xgb_accuracy:.4f}\n")
    f.write(f"RandomForest Accuracy: {rf_accuracy:.4f}\n")

# Save classification reports to files
with open(os.path.join(result_dir, "xgboost_report.txt"), "w") as f:
    f.write("XGBoost Classification Report:\n")
    f.write(xgb_report)
with open(os.path.join(result_dir, "randomforest_report.txt"), "w") as f:
    f.write("RandomForest Classification Report:\n")
    f.write(rf_report)

# Plot accuracy metrics
plt.figure(figsize=(8, 6))
models = ['XGBoost', 'RandomForest']
accuracies = [xgb_accuracy, rf_accuracy]
plt.bar(models, accuracies, color=['blue', 'green'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # Set y-axis limit from 0 to 1
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center')
plt.savefig(os.path.join(result_dir, "accuracy_comparison.png"))
plt.close()
print(f"Graph saved as {os.path.join(result_dir, 'accuracy_comparison.png')}")